### Imports

In [1]:
import os
import random
import functools
import cv2
import IPython.display as display
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Paths
from chestxray.config import (PANDA_PATH,
                              MODEL_PATH,
                              PANDA_IMGS,
                              PANDA_MASKS,
                              TRAIN_CSV)
# Competition related config
from chestxray.config import CFG

from chestxray.datasets import img_to_tiles

#### Global Constants

### Retrieve the Images

In [3]:
ls -la {PANDA_PATH}/

total 3172
drwxrwxr-x 5 ademyanchuk ademyanchuk   4096 Jun  4 18:00 ./
drwxr-xr-x 8 ademyanchuk ademyanchuk   4096 Apr 28 13:53 ../
-rw-rw-r-- 1 ademyanchuk ademyanchuk 517985 Jun  4 18:00 folds_cleaned.csv
-rw-rw-r-- 1 ademyanchuk ademyanchuk 541440 May  4 22:31 folds.csv
-rw-rw-r-- 1 ademyanchuk ademyanchuk   5140 May  6 19:15 folds_db.csv
drwxrwxr-x 2 ademyanchuk ademyanchuk   4096 May 28 08:16 hdf5/
-r--r--r-- 1 ademyanchuk ademyanchuk  29380 Jun  3 17:15 PANDA_Suspicious_Slides.csv
-rw-rw-r-- 1 ademyanchuk ademyanchuk    125 Apr 22 02:10 sample_submission.csv
-rw-rw-r-- 1 ademyanchuk ademyanchuk    149 Apr 22 02:10 test.csv
-rw-rw-r-- 1 ademyanchuk ademyanchuk 520203 Apr 22 02:10 train.csv
drwxrwxr-x 2 ademyanchuk ademyanchuk 790528 May 23 10:56 train_images/
drwxrwxr-x 2 ademyanchuk ademyanchuk 802816 Apr 28 23:33 train_label_masks/


In [4]:
PANDA_IMGS

PosixPath('/home/ademyanchuk/Projects/ChestXRay/data/Panda/train_images')

In [5]:
TRAIN_DF = pd.read_csv(TRAIN_CSV)

In [6]:
TRAIN_DF.head()

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


### Permute Hard Classes

In [7]:
tmp = TRAIN_DF.copy()

In [13]:
hard_cls_values = tmp.loc[tmp.isup_grade.isin([2,3,4]), "isup_grade"].copy().values

In [16]:
hard_cls_values[:5]

array([4, 4, 3, 3, 3])

In [17]:
np.random.shuffle(hard_cls_values)

In [18]:
hard_cls_values[:5]

array([3, 2, 3, 3, 4])

In [20]:
tmp.loc[tmp.isup_grade.isin([2,3,4]), "isup_grade"] = hard_cls_values

In [22]:
(tmp.loc[tmp.isup_grade.isin([2,3,4]), "isup_grade"] == TRAIN_DF.loc[TRAIN_DF.isup_grade.isin([2,3,4]), "isup_grade"]).mean()

0.32968179447052687